<a href="https://colab.research.google.com/github/qwert-top/NMM-recognition-with-OpenPose/blob/main/json_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import numpy as np
import glob
import sys

In [ ]:
fP = 0.5    #確度のデフォルト値

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
InputPath = "/content/drive/MyDrive/proB/何が食べたいですか？/output"
OutputPath = "/content/drive/MyDrive/proB/何が食べたいですか？"

In [ ]:
json_list = glob.glob(InputPath + "/*.json")
ilenlist = len(json_list)

In [ ]:
#各点の名前を指定
#各点の名前を指定
cols = []
for i in range(210):
  if (i%3 == 0):
    ans = int(i/3)
    cols.append("{ans}_x".format(ans=ans))
  elif (i%3 == 1):
    ans = int((i-1)/3)
    cols.append("{ans}_y".format(ans=ans))
  elif (i%3 == 2):
    ans = int((i-2)/3)
    cols.append("P{ans}".format(ans=ans))

print(cols)
dfsum = pd.DataFrame(index=[],columns=cols)

In [ ]:
#face_keypointsのjsonファイルを読み込んで結合する
for i,file in enumerate(json_list):
  with open(file) as f:
    data = json.load(f)
    data = np.array(data['people'][0]['face_keypoints_2d']).reshape(-1,210)
    # 3成分(X,Y,P)×25マーカ
  df = pd.DataFrame(data,columns=cols)
  #「確度」の値によって、XYの値をNULLにする
  for num in range(2, 210, 3):
    if df.iat[0,num]<fP:
      df.iat[0,num-1]=np.nan
      df.iat[0,num-2]=np.nan
   #結合
  dfsum = dfsum.append(df, ignore_index=True)
  #進捗バー
  dPercent = (i+1)/ilenlist*100
  print("\r{0}% [{1}]".format(int(dPercent), "#" * int(dPercent)), end="")

#出力する
dfsum.to_csv(OutputPath + '/face_out.csv')
